### Imports & Preparation

In [1]:
import sys
sys.path.insert(1, '../')  # to load from any submodule in the repo

from models3D import detector3D

import time
import gc
import torch

device = torch.device("cpu")

if torch.device("cuda"):
    device = torch.device("cuda")
    print("Using", torch.cuda.device_count(), "CUDA devices")

seed_file = open('../utils/seed.txt', "r")
seed = int(seed_file.read())
seed_file.close()

print ("Using Seed:", seed)
    
_ = torch.manual_seed(seed)

Using 1 CUDA devices
Using Seed: 34897567


In [8]:
print(torch.finfo(torch.float32).eps)

1.1920928955078125e-07


In [2]:
def knn(x, k):
    
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    
    torch.cuda.empty_cache()
    
    return idx

In [3]:
def get_graph_feature(x, k=20, idx=None, dim9=False):
    batch_size = x.size(0)
    num_points = x.size(2)

    x = x.view(batch_size, -1, num_points)
    if idx is None:
        if dim9 == False:
            idx = knn(x, k=k)   # (batch_size, num_points, k)
        else:
            idx = knn(x[:, 6:], k=k)
    device = torch.device('cuda')

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2)
    
    torch.cuda.empty_cache()
  
    return feature      # (batch_size, 2*num_dims, num_points, k)

In [4]:
N = 20

detector = detector3D.Model(k = 20, emb_dims = 1024, dropout = 0.5).to(device)

tlist = [torch.rand(1,3,5000).to(device) for i in range(N)]

start = time.time()

for i in range(N):
    
    batch = 0
    
    for b in range(10):
    
        batch += detector(tlist[i])
        
    torch.cuda.empty_cache()
    

end = time.time()

print (end - start, "sec")

10.203725814819336 sec


In [6]:
del detector
del tlist
del batch
torch.cuda.empty_cache()